https://blog.csdn.net/qq_24951479/article/details/132495408

第十一周作业：
1. 参考课堂案例，使用指定的数据集，编写代码实现ner模型训练和推流。
https://huggingface.co/datasets/doushabao4766/msra_ner_k_V3
2. 完成预测结果的实体抽取。
    输入：“双方确定了今后发展中美关系的指导方针。”
    输出：[{"entity":"ORG","content":"中"},{"entity":"ORG","content":"美"}]
3. 整理Dataset、Trainer、TrainingArgument、DataCollator、Evaluate 知识点，总结文档（无需提交）

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jd_comment_with_label/jd_comment_data.xlsx


# 导入相关包

In [2]:
import torch
import torch.nn as nn
import pandas as pd
from tqdm import tqdm
from torch.optim import Adam  # 优化器
from transformers import AutoTokenizer , AutoModelForSequenceClassification  # 分词器
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter #Tensorboard 跟踪
from sklearn.model_selection import train_test_split # 自动拆分测试集和验证集



2025-05-31 11:50:19.471785: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748692219.924268      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748692220.052600      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 数据预处理

In [3]:
# 读取表格数据
data = pd.read_excel('/kaggle/input/jd_comment_with_label/jd_comment_data.xlsx')

# data.head()  # 查看读取出来的数据

# data.columns  # 查看所有的列名

data_content = data['评价内容(content)'] != '此用户未填写评价内容'  # 获取评价内容，并且存入data_content中,去掉'此用户未填写评价内容'
data_two = data[data_content][['评价内容(content)' , '评分（总分5分）(score)']]  # 将评价内容和评分提取出来


data_ts = data_two.values  # 将数据转换为数组
train , test = train_test_split(data_ts)

print(train.shape)

(33316, 2)


# 创建分词器以及将数据转换成DataLoader

In [4]:
# 分词器
tokenzier = AutoTokenizer.from_pretrained('hfl/chinese-bert-wwm')

# 创建自定义DataLoader方法 collatefron = 
def warp_data(btach_data):
    comments , lables = [] , []
    for b in btach_data:
        comments.append(b[0])
        lables.append(int(b[1])-1)  # 1-5分 所以标签取值0-4

    # 转换成模型输入的数据
    input_data = tokenzier(comments , return_tensors = 'pt' , # 返回ptorch类型的数据
                          padding = True , # 自动补全
                          truncation = True , # 超过最大值512 允许截断
                          max_length = 512)
    input_labels = torch.tensor(lables)
    return input_data , input_labels

# 装换成DataLoader
train_dl = DataLoader(train , batch_size = 20 , shuffle = True , collate_fn = warp_data)
test_dl = DataLoader(test , batch_size = 20 , shuffle = False , collate_fn = warp_data)

for item in test_dl:
    print(item)
    break

tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

({'input_ids': tensor([[ 101, 3418, 3315, 2218,  679,  833, 4178, 8024, 2697, 6230, 1008,  137,
         4638, 8024, 2145, 3302, 2523, 4178, 2658,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0],
        [ 101,  691, 6205,  679, 7231, 1557, 8024, 3173, 2399, 2571,  727, 8024,
          674,  752, 1963, 2692,  511,  809, 1400, 1914, 1914,  912, 2139,  102,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0],
        [ 101, 6820, 1377,  809, 1416, 8024, 1057, 2797,  738,  679, 3221, 6929,
         1408, 2141, 4500,  102,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0],
        [ 101, 2644, 3766, 3300, 1856, 1091, 1079, 2159, 8024, 7949, 6371, 1962,
         6397,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          

# 构建模型（模型微调 和 迁移学习bert）

In [5]:
# 选择设备
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# tensor 跟踪loss
writer = SummaryWriter()


#构建模型
model_1 = AutoModelForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm' , num_labels = 5)
model_2 = AutoModelForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm' , num_labels = 5)

model_1 = model_1.to(device)
model_2 = model_2.to(device)
# 冻结bert
model_2.bert.trainable = False


pytorch_model.bin:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-bert-wwm and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-bert-wwm and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 构建优化器和损失

In [6]:
# 损失
loss_fn1 = nn.CrossEntropyLoss()
optim1 = Adam(model_1.parameters() , lr = 1e-4)

loss_fn2 = nn.CrossEntropyLoss()
optim2 = Adam(model_2.parameters() , lr = 1e-4)

# 训练不冻结模型

In [7]:
# model1
model1_train_loss_cnt = 0
for epoch in range(5):
    model_1.train()
    pbar = tqdm(train_dl)
    for input_data , input_labels in pbar:
        datas = { k:v.to(device) for k , v in input_data.items()}
        labels = input_labels.to(device)

        result = model_1(**datas)
        loss = loss_fn1(result.logits , labels)

        pbar.set_description(f'epoch:{epoch} train_loss:{loss.item():.4f}' )
        writer.add_scalar("Fine Tuning Tain Loss", loss , model1_train_loss_cnt)

        model1_train_loss_cnt +=1

        loss.backward()
        optim1.step()
        model_1.zero_grad()


torch.save(model_1.state_dict() , 'model_1.pt')
    

epoch:4 train_loss:0.0547: 100%|██████████| 1666/1666 [11:41<00:00,  2.38it/s]


# 训练冻结bert模型

In [8]:
model2_train_loss_cnt = 0
for epoch in range(5):
    model_2.train()
    pbar = tqdm(train_dl)
    for input_data , labels_data in pbar:
        datas = { k:v.to(device) for k,v in input_data.items()}
        labels = labels_data.to(device)

        result = model_2(**datas)
        loss = loss_fn2(result.logits , labels)

        pbar.set_description(f'epoch:{epoch} train_loss:{loss.item():.4f}')
        writer.add_scalar("Transfer Learning Train Loss", loss, model2_train_loss_cnt)
        model2_train_loss_cnt += 1
        
        loss.backward()
        optim2.step()
        
        model_2.zero_grad()

torch.save(model_2.state_dict() , 'model_2.pt')


epoch:4 train_loss:0.0514: 100%|██████████| 1666/1666 [11:42<00:00,  2.37it/s]


# 评估模型

In [9]:
model_1.eval()
model_2.eval()
pbar = tqdm(test_dl)
correct1 , correct2 = 0 , 0

for input_data, labels_data in pbar:
    datas = { k:v.to(device) for k,v in input_data.items() }
    labels = labels_data.to(device)

    with torch.no_grad():
        result1 = model_1(**datas)
        result2 = model_2(**datas)

    predict1 = torch.argmax(result1.logits, dim=-1)
    predict2 = torch.argmax(result2.logits, dim=-1)

    correct1 += (predict1 == labels).sum()
    correct2 += (predict1 == labels).sum()


100%|██████████| 556/556 [02:30<00:00,  3.71it/s]


准确率 model1:0.0005719187902286649%.4f
准确率 model2:0.0005719187902286649%.4f
